In [ ]:
# Uploading Dataset from Kaggle
from google.colab import files
files.upload()

import os
os.makedirs('/root/.kaggle', exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d benedictusjason/nthuddd2

!unzip nthuddd2.zip -d /content/

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/benedictusjason/nthuddd2
License(s): unknown
 99% 2.79G/2.81G [00:27<00:00, 147MB/s]
100% 2.81G/2.81G [00:27<00:00, 110MB/s]


A

In [ ]:
# Importing Important Library
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Processing Downloaded Dataset
train_dir = '/content/dataset/images/train/'
validation_dir = '/content/dataset/images/val/'

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 52960 images belonging to 2 classes.
Found 13000 images belonging to 2 classes.


In [ ]:
# Preparing pre-trained model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)  # Adjust for the number of classes

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Training Model with prepared dataset
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10  # Adjust the number of epochs as needed
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1655/1655 ━━━━━━━━━━━━━━━━━━━━ 772s 458ms/step - accuracy: 0.7201 - loss: 0.5595 - val_accuracy: 0.8531 - val_loss: 0.3327
Epoch 2/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1655/1655 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7500 - val_loss: 0.4775
Epoch 3/10
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 799s 462ms/step - accuracy: 0.8181 - loss: 0.3833 - val_accuracy: 0.8521 - val_loss: 0.3280
Epoch 4/10
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 0s 21us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7500 - val_loss: 0.4963
Epoch 5/10
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 790s 455ms/step - accuracy: 0.8413 - loss: 0.3398 - val_accuracy: 0.8706 - val_loss: 0.2917
Epoch 6/10
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 0s 62us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.3750 - val_loss: 0.9323
Epoch 7/10
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 826s 470ms/step - accuracy: 0.8529 - loss: 0.3187 - val_accuracy: 0.8728 - val_loss: 0.2754
Epoch 8/10
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 0s 116us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8750 - val_loss: 0.2855
Epoch 9/10
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 842s 494ms/

In [ ]:
# Model Evaluation
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {accuracy * 100:.2f}%')


407/407 ━━━━━━━━━━━━━━━━━━━━ 45s 110ms/step - accuracy: 0.8937 - loss: 0.2568
Validation accuracy: 89.30%


In [ ]:
# Saving Model
model.save('mobilenet_awake_drowsy.keras')

In [ ]:
from tensorflow.keras.models import load_model

# Load the trained MobileNet model
model = load_model('mobilenet_awake_drowsy.keras')


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Function to load and preprocess the image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))  # Resize image
    img_array = image.img_to_array(img)  # Convert to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize the image
    return img_array


In [ ]:
# Testing trained model with test pictures
img_path = 'test.jpg'

test_image = preprocess_image(img_path)

predictions = model.predict(test_image)

predicted_class = np.argmax(predictions, axis=-1)

class_labels = ['awake', 'drowsy']
predicted_label = class_labels[predicted_class[0]]

print(f"Predicted class: {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted class: awake


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

model = load_model('mobilenet_awake_drowsy.keras')

cap = cv2.VideoCapture(0)  # Use 0 for the default camera

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img = cv2.resize(frame, (224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=-1)

    class_labels = ['awake', 'drowsy']
    predicted_label = class_labels[predicted_class[0]]

    cv2.putText(frame, f'Prediction: {predicted_label}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Webcam Feed', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.10.0) /io/opencv/modules/highgui/src/window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
